In [1]:
import netCDF4 as nc
from netCDF4 import Dataset
import xarray as xr
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import math
import plotly.graph_objects as go

## User Inputs:

In [2]:
# Open NETCDF file
ds = xr.open_dataset(r"C:\Users\Felicio.Cassalho\JupyterLab_Codes\SCHISM\schism_3day_combined\schism_3day\outputs\schism_3day_combined_1.nc")
# Select the Layers to be plotted
layers = [-5]#[0,1,2,3,4,6,8,11,14,18,22,-4,-1]
# Select the time steps to be plotted
time = [0,6,12,18,24,30,36,42,48,54,60,66,-1]#[0,6,12,18,24,30,36,42,48,54,60,66]#[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
# Select the variable to be plotted
variable = 'currents'#currents, #temp
# Select if bathymetry will be plotted
add_bath = "yes" #"yes","no"

C:\Users\Felicio.Cassalho\Anaconda3\envs\pyfvcom\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [3]:
ds

<xarray.Dataset>
Dimensions:                  (time: 72, one: 1, nSCHISM_hgrid_face: 64453,
                              nMaxSCHISM_hgrid_face_nodes: 4,
                              nSCHISM_hgrid_edge: 98855, two: 2,
                              nSCHISM_hgrid_node: 34395, sigma: 30,
                              nSCHISM_vgrid_layers: 30)
Coordinates:
  * time                     (time) datetime64[ns] 2018-05-05T01:00:00 ... 20...
  * sigma                    (sigma) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Dimensions without coordinates: one, nSCHISM_hgrid_face,
                                nMaxSCHISM_hgrid_face_nodes,
                                nSCHISM_hgrid_edge, two, nSCHISM_hgrid_node,
                                nSCHISM_vgrid_layers
Data variables: (12/41)
    SCHISM_hgrid             (one) int32 ...
    SCHISM_hgrid_face_nodes  (nSCHISM_hgrid_face, nMaxSCHISM_hgrid_face_nodes) float64 ...
    SCHISM_hgrid_edge_nodes  (nSCHISM_hgrid_edge, two) int32 ...
    SCHISM_hgrid_node_x      (nSCHISM_hgrid_node) float64 ...
    SCHISM_hgrid_node_y      (nSCHISM_hgrid_node) float64 ...
    node_bottom_index        (nSCHISM_hgrid_node) int32 ...
    ...                       ...
    evaporation              (time, nSCHISM_hgrid_node) float32 ...
    precipitation            (time, nSCHISM_hgrid_node) float32 ...
    wind_speed               (time, nSCHISM_hgrid_node, two) float32 ...
    dahv                     (time, nSCHISM_hgrid_node, two) float32 ...
    temp                     (time, nSCHISM_hgrid_node, nSCHISM_vgrid_layers) float32 ...
    hvel                     (time, nSCHISM_hgrid_node, nSCHISM_vgrid_layers, two) float32 ...
Attributes:
    Conventions:   CF-1.0, UGRID-1.0
    title:         SCHISM Model output
    institution:   SCHISM Model output
    source:        SCHISM model output version v10
    references:    http://ccrm.vims.edu/schismweb/
    history:       created by combine_output11
    comment:       SCHISM Model output
    type:          SCHISM Model output
    VisIT_plugin:  https://schism.water.ca.gov/library/-/document_library/vie...

#### Processing

In [10]:
x, y = np.array(ds['SCHISM_hgrid_node_x'])+360, np.array(ds['SCHISM_hgrid_node_y'])
triangles=np.array(ds['SCHISM_hgrid_face_nodes'][:]-1)
triangles1=np.array([[int(x) for x in y if not np.isnan(x)] for y in triangles])

In [13]:
triangles1

array([[    1,    13,     0],
       [   12,     0,    13],
       [   11,     0,    12],
       ...,
       [34393, 34391, 34389],
       [34394, 34391, 34393],
       [34392, 34391, 34394]])

In [7]:
x

array([280.2003892, 280.2006994, 280.2010539, ..., 282.4012718,
       282.4028066, 282.401065 ])

#### Creating frames

In [4]:
fig_dict = {
    "data": [],
    # "layout": {},
    "frames": []
}
# make figure
if variable == 'currents':
    values = (np.array(ds['hvel'][:,:,:,0])**2+np.array(ds['hvel'][:,:,:,1])**2)**.5
    cmax,cmin = values[~np.isnan((np.array(ds['hvel'][:,:,:,0])**2+np.array(ds['hvel'][:,:,:,1])**2)**.5)].max(), values[~np.isnan((np.array(ds['hvel'][:,:,:,0])**2+np.array(ds['hvel'][:,:,:,1])**2)**.5)].min()
    # make data
    for l in layers:
        data_dict = {
            "x": np.array(x, dtype="float32"),
            "y": np.array(y, dtype="float32"),
            "z": np.array(ds['zcor'][0,:,l]),
            "i": np.array(triangles[:,0]),
            "j": np.array(triangles[:,1]),
            "k": np.array(triangles[:,2]),
            "intensity":(np.array(ds['hvel'][0,:,l,0])**2+np.array(ds['hvel'][0,:,l,1])**2)**.5,
            #"intensitymode":"cell",
            "cmax":cmax,#.2,
            "cmin":cmin,#0,
            "colorscale":'rainbow',
            #"coloraxis":"coloraxis",
            "opacity":.6,
        }
        data_dict['z'][data_dict['z']>100]= np.nan
        data_dict['intensity'][data_dict['intensity']>100]= np.nan
        fig_dict["data"].append(data_dict)
        
    if add_bath == "yes":
        bath = {
            "x": np.array(x),
            "y": np.array(y),
            "z": np.array(ds['depth']*-1),
            "i": np.array(triangles[:,0]),
            "j": np.array(triangles[:,1]),
            "k": np.array(triangles[:,2]),
            #"intensity":np.array(ds['depth']*-1),
            #"intensitymode":"cell",
            # "cmax":.2,
            # "cmin":-100,
            "color":"gray",
            "showscale": False,
            #"vertexcolor" : "black",
            #"colorscale":'YlOrBr',
            #"showlegend": False,
            #"coloraxis":"coloraxis1",
            "opacity":.1,
        }
        fig_dict["data"].append(bath)     

    for t in time:
        frame = {"data": [], "name": str(pd.DatetimeIndex(ds['time'].values)[t])}   
        for l in layers:
            frame_dict = {
                "x": np.array(x),
                "y": np.array(y),
                "z": np.array(ds['zcor'][t,:,l]),
                "i": np.array(triangles[:,0]),
                "j": np.array(triangles[:,1]),
                "k": np.array(triangles[:,2]),
                "intensity":(np.array(ds['hvel'][t,:,l,0])**2+np.array(ds['hvel'][t,:,l,1])**2)**.5,
                #"intensitymode":"cell",
                "cmax":cmax,#.2,
                "cmin":cmin,#0,
                "colorscale":'rainbow',
                #"coloraxis":"coloraxis",
                "opacity":.6,
            }
            frame_dict['z'][frame_dict['z']>100]= np.nan
            frame_dict['intensity'][frame_dict['intensity']>100]= np.nan
            frame["data"].append(frame_dict)
            
        if add_bath == "yes":
            bath = {
                "x": np.array(x),
                "y": np.array(y),
                "z": np.array(ds['depth']*-1),
                "i": np.array(triangles[:,0]),
                "j": np.array(triangles[:,1]),
                "k": np.array(triangles[:,2]),
                #"intensity":np.array(ds['depth']*-1),
                #"intensitymode":"cell",
                "color":"gray",
                # "cmax":.2,
                # "cmin":-100,
                #"vertexcolor" : "black",
                #"colorscale":'YlOrBr_r',
                "showscale": False,
                #"coloraxis":"coloraxis1",
                "opacity":.1,
            }

            frame["data"].append(bath)     
            
        fig_dict["frames"].append(frame)       

        
else:
    values = np.array(ds['temp'][:,:,:])
    cmax,cmin = values[~np.isnan(np.array(ds['temp'][:,:,:]))].max(), values[~np.isnan(np.array(ds['temp'][:,:,:]))].min()
    # make data
    for l in layers:
        data_dict = {
            "x": np.array(x, dtype="float32"),
            "y": np.array(y, dtype="float32"),
            "z": np.array(ds['zcor'][0,:,l]),
            "i": np.array(triangles[:,0]),
            "j": np.array(triangles[:,1]),
            "k": np.array(triangles[:,2]),
            "intensity":np.array(ds['{}'.format(variable)][0,:,l]),
            #"intensitymode":"cell",
            "cmax":cmax,
            "cmin":cmin,
            "colorscale":'rainbow',
            #"coloraxis":"coloraxis",
            "opacity":.6,
        }
        data_dict['z'][data_dict['z']>100]= np.nan
        data_dict['intensity'][data_dict['intensity']>100]= np.nan
        fig_dict["data"].append(data_dict)
    if add_bath == "yes":
        bath = {
            "x": np.array(x),
            "y": np.array(y),
            "z": np.array(ds['depth']*-1),
            "i": np.array(triangles[:,0]),
            "j": np.array(triangles[:,1]),
            "k": np.array(triangles[:,2]),
            #"intensity":np.array(ds['depth']*-1),
            #"intensitymode":"cell",
            # "cmax":.2,
            # "cmin":-100,
            "color":"gray",
            "showscale": False,
            #"vertexcolor" : "black",
            #"colorscale":'YlOrBr',
            #"showlegend": False,
            #"coloraxis":"coloraxis1",
            "opacity":.1,
        }
        fig_dict["data"].append(bath)     
        
    # make frames
    for t in time:
        frame = {"data": [], "name": str(pd.DatetimeIndex(ds['time'].values)[t])}   
        for l in layers:
            frame_dict = {
                "x": np.array(x),
                "y": np.array(y),
                "z": np.array(ds['zcor'][t,:,l]),
                "i": np.array(triangles[:,0]),
                "j": np.array(triangles[:,1]),
                "k": np.array(triangles[:,2]),
                "intensity":np.array(ds['{}'.format(variable)][t,:,l]),
                #"intensitymode":"cell",
                "cmax":cmax,
                "cmin":cmin,
                "colorscale":'rainbow',
                #"coloraxis":"coloraxis",
                "opacity":.6,
            }
            frame_dict['z'][frame_dict['z']>100]= np.nan
            frame_dict['intensity'][frame_dict['intensity']>100]= np.nan
            frame["data"].append(frame_dict)
        if add_bath == "yes":
            bath = {
                "x": np.array(x),
                "y": np.array(y),
                "z": np.array(ds['depth']*-1),
                "i": np.array(triangles[:,0]),
                "j": np.array(triangles[:,1]),
                "k": np.array(triangles[:,2]),
                #"intensity":np.array(ds['depth']*-1),
                #"intensitymode":"cell",
                "color":"gray",
                # "cmax":.2,
                # "cmin":-100,
                #"vertexcolor" : "black",
                #"colorscale":'YlOrBr_r',
                "showscale": False,
                #"coloraxis":"coloraxis1",
                "opacity":.1,
            }

            frame["data"].append(bath)     
            
        fig_dict["frames"].append(frame)   
        

    

### Creating Plot

In [5]:
if add_bath == "yes":
    fig=go.Figure(data=[go.Mesh3d(fig_dict['data'][l]) for l in range(len(layers)+1)], #This +1 is needed if bath is added
                  layout=go.Layout(xaxis=dict(range=[0, 5], autorange=False),
                                   yaxis=dict(range=[0, 5], autorange=False),
                                   title=variable,
                                   updatemenus=[dict(type="buttons",buttons=[dict(label="Play",method="animate",args=[None])])]),
                  frames = [go.Frame(data=[go.Mesh3d(fig_dict['frames'][t]['data'][l]) for l in range(len(layers)+1)], name = f'fr{t}') for t in range(len(time))]) #This +1 is needed if bath is added
elif add_bath == "no":
    fig=go.Figure(data=[go.Mesh3d(fig_dict['data'][l]) for l in range(len(layers))],
                  layout=go.Layout(xaxis=dict(range=[0, 5], autorange=False),
                                   yaxis=dict(range=[0, 5], autorange=False),
                                   title=variable,
                                   updatemenus=[dict(type="buttons",buttons=[dict(label="Play",method="animate",args=[None])])]),
                  frames = [go.Frame(data=[go.Mesh3d(fig_dict['frames'][t]['data'][l]) for l in range(len(layers))], name = f'fr{t}') for t in range(len(time))])

sliders =  [dict(steps= [dict(method= 'animate',
                              args= [[f'fr{t}'], #this steps refers to the frame of name f'fr{k}
                                      dict(mode= 'immediate',
                                           frame= dict(duration=300, redraw= True ),
                                           fromcurrent=True,
                                           transition=dict(duration= 0))],
                                      label=str(pd.DatetimeIndex(ds['time'].values)[t])) for t  in time], #this is the step label on the slider 
                                      minorticklen=0,
                                      x=0,
                                   len=1)]
fig.update_layout(sliders=sliders)

# fig.write_html('first_figure_{}.html'.format(variable), auto_open=True)
fig.show()